### 실습
- 단어 맞추기 프로그램 만들기
- 밀크티초등 쏙쏙 words의 fly-1번 단어장 데이터를 참조하여 단어 맞추기 프로그램을 만들어 보세요.
- 사용자 입력과 정답 사이의 코사인 유사도를 파악한 뒤, 얼마나 비슷한지를 리턴하도록 합시다.

In [1]:
font_path = "./Noto_Sans_KR/NotoSansKR-VariableFont_wght.ttf"
csv_file = './data/full_vocabulary_lesson.csv'

In [2]:
import pandas as pd

df = pd.read_csv(csv_file)
df

,단어,단어 뜻,예문,예문 뜻
0,bird,새,A bird is in the tree.,새가 나무에 있다.
1,idea,생각,Matt has a good idea.,맷에게 좋은 생각이 있다.
2,wing,날개,Chickens have wings.,닭은 날개를 가지고 있다.
3,wind,바람,The wind was strong.,바람이 강했다.
4,car,자동차,Is this your car?,이것은 네 차니?
5,sky,하늘,Look at the sky.,하늘을 봐.
6,airplane,비행기,The airplane is in the sky.,비행기가 하늘에 있다.
7,cousin,사촌,My cousin is kind.,내 사촌은 친절하다.
8,notebook,공책,Do you have a new notebook?,너는 새 공책이 있니?
9,vacation,방학,I like summer vacation.,나는 여름 방학이 좋다.


In [3]:
import pygame
import random
import sys
import time

from sentence_transformers import SentenceTransformer, util

# 한국어 유사도 계산 함수
def calculate_similarity(model, selected_word_meaning, correct_meaning):
    input_embedding = model.encode(selected_word_meaning)
    correct_embedding = model.encode(correct_meaning)
    similarity = util.pytorch_cos_sim(input_embedding, correct_embedding).item()
    return similarity

# CSV 파일에서 데이터를 불러오기
def load_quiz_data(csv_file):
    df = pd.read_csv(csv_file)
    quiz_data = []
    for index, row in df.iterrows():
        sentence_translation = row['예문 뜻']
        word_meaning = row['단어 뜻']
        word = row['단어']
        
        # 단어 뜻과 유사한 단어를 찾아서 빈칸으로 처리하는 함수 사용
        sentence_with_blank = replace_with_blank(sentence_translation, word_meaning)

        quiz_data.append({
            "word": word,  # 힌트로 제공할 영어 단어
            "word_meaning": word_meaning,  # 사용자가 맞춰야 할 단어 뜻
            "sentence_translation": sentence_with_blank  # 유사도 기반으로 빈칸 처리된 예문 뜻
        })
    return quiz_data

# 문장에서 단어 뜻과 가장 유사한 단어를 빈칸으로 교체하는 함수
def replace_with_blank(sentence, word_meaning):
    # 한국어 조사 목록
    korean_postpositions = ['이구나','은', '는', '이', '가', '을', '를', '와', '과', '로', '으로', '에서', '에게', '한테', '께', '에서', '에게서', '한테서', '께서', '에서부터', '까지', '도', '만', '조차', '마저', '까지도', '부터', '의', '으로서', '로써', '처럼', '같이', '만큼', '밖에', '마냥', '커녕', '보다', '하고']

    words_in_sentence = sentence.split()  # 문장을 단어로 분할
    max_similarity = 0
    target_word = None
    
    # 문장 내 각 단어와 단어 뜻의 유사도를 계산
    for word in words_in_sentence:
        # 단어에서 조사 제거
        for postpos in korean_postpositions:
            if word.endswith(postpos):
                word = word[:-len(postpos)]
                break

        similarity = calculate_similarity(model, word, word_meaning)
        if similarity > max_similarity:
            max_similarity = similarity
            target_word = word
    
    # 유사도가 높은 단어를 빈칸으로 대체
    if target_word:
        for postpos in korean_postpositions:
            if target_word.endswith(postpos):
                target_word = target_word[:-len(postpos)]
                break
        sentence_with_blank = sentence.replace(target_word + postpos, '___' + postpos) if postpos in target_word else sentence.replace(target_word, '___')
    else:
        sentence_with_blank = sentence  # 유사한 단어가 없으면 변경하지 않음
    
    return sentence_with_blank

# SentenceTransformer 모델 로드 (한국어 유사도 계산용)
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# pygame 초기화
pygame.init()

# 화면 크기 설정
screen_width, screen_height = 800, 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("빈칸 맞추기 퀴즈 게임")

# 폰트 설정 (시스템에 맞는 한국어 폰트 경로를 넣어주세요)

font = pygame.font.Font(font_path, 50)
small_font = pygame.font.Font(font_path, 30)

# 색상 정의
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
BLUE = (100, 149, 237)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# 퀴즈 데이터 로드 (CSV 파일에서 불러옴)

quiz_data = load_quiz_data(csv_file)

# 텍스트 렌더링 함수
def draw_text(text, font, color, surface, x, y):
    text_obj = font.render(text, True, color)
    text_rect = text_obj.get_rect(center=(x, y))
    surface.blit(text_obj, text_rect)



# 선택지 섞기
def shuffle_options(correct_meaning, all_words):
    options = [correct_meaning] + random.sample([word["word_meaning"] for word in all_words if word["word_meaning"] != correct_meaning], 2)
    random.shuffle(options)
    return options

# 결과창 함수 수정
def show_results(score, total_questions_attempted):
    screen.fill(WHITE)
    draw_text(f"최종 점수: {score}/{total_questions_attempted}", font, BLACK, screen, screen_width // 2, screen_height // 2)
    pygame.display.update()
    pygame.time.wait(3000)  # 3초 대기 후 종료
    pygame.quit()
    sys.exit()

# 게임 시작 화면
def start_screen():
    screen.fill(WHITE)
    draw_text("빈칸 맞추기 퀴즈 게임", font, BLACK, screen, screen_width // 2, screen_height // 3)
    draw_text("시작하려면 ENTER를 누르세요", small_font, BLUE, screen, screen_width // 2, screen_height // 2)
    pygame.display.update()

    waiting = True
    while waiting:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                waiting = False

# 퀴즈 진행 함수
def play_quiz():
    running = True
    score = 0
    total_questions_attempted = 0  # 실제로 푼 문제 수
    remaining_data = quiz_data.copy()
    current_quiz = random.choice(remaining_data)
    options = shuffle_options(current_quiz["word_meaning"], quiz_data)
    user_choice = 0
    correct = False
    show_answer = False
    start_time = time.time()
    time_limit = 60  # 60초 제한

    while running:
        elapsed_time = time.time() - start_time
        if elapsed_time > time_limit:
            show_results(score, total_questions_attempted)  # 결과창 수정

        screen.fill(WHITE)
        draw_text(current_quiz["sentence_translation"], font, BLACK, screen, screen_width // 2, screen_height // 4)
        draw_text(f"힌트: {current_quiz['word']}", small_font, BLUE, screen, screen_width // 2, screen_height // 4 + 60)

        for i, option in enumerate(options):
            color = GREEN if i == user_choice else BLACK
            draw_text(option, small_font, color, screen, screen_width // 2, screen_height // 2 + i * 40)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_UP:
                    user_choice = (user_choice - 1) % 3
                elif event.key == pygame.K_DOWN:
                    user_choice = (user_choice + 1) % 3
                elif event.key == pygame.K_RETURN:
                    selected_meaning = options[user_choice]
                    similarity = calculate_similarity(model, selected_meaning, current_quiz["word_meaning"])
                    total_questions_attempted += 1  # 푼 문제 수 업데이트

                    if similarity >= 0.99:
                        correct = True
                        score += 1
                    else:
                        correct = False
                    show_answer = True

        if show_answer:
            remaining_data.remove(current_quiz)  # 중복 제거
            if correct:
                draw_text("정답입니다!", font, GREEN, screen, screen_width // 2, screen_height - 100)
            else:
                draw_text(f"틀렸습니다. 정답은 '{current_quiz['word_meaning']}'입니다.", font, RED, screen, screen_width // 2, screen_height - 100)

            draw_text(f"유사도: {similarity:.2f}", small_font, BLACK, screen, screen_width // 2, screen_height - 50)
            pygame.display.update()
            pygame.time.wait(1000)  # 1초 대기 후 다음 문제

            if remaining_data:
                current_quiz = random.choice(remaining_data)
                options = shuffle_options(current_quiz["word_meaning"], quiz_data)
                user_choice = 0
                show_answer = False
            else:
                show_results(score, total_questions_attempted)

        remaining_time = int(time_limit - elapsed_time)
        draw_text(f"남은 시간: {remaining_time}초", small_font, BLACK, screen, screen_width // 2, screen_height - 50)
        pygame.display.update()

# 게임 실행
start_screen()  # 게임 시작 화면 표시
play_quiz()  # 게임 진행

pygame 2.6.1 (SDL 2.28.4, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


SystemExit: 

d:\workspace\Chunjae_Bigdata_9th\9.4~9.23\.conda\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
